In [1]:
from __future__ import print_function, division

from keras.models import Sequential, Model
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal
from keras.applications import *
import keras.backend as K
from tensorflow.contrib.distributions import Beta
import tensorflow as tf
from keras.optimizers import Adam
from keras import losses
from keras.layers.merge import _Merge

Using TensorFlow backend.


In [2]:
from image_augmentation import random_transform
from image_augmentation import random_warp
from utils import get_image_paths, load_images, stack_images
from pixel_shuffler import PixelShuffler

In [3]:
import time
import numpy as np
from PIL import Image
import cv2
import glob
from random import randint, shuffle
from IPython.display import clear_output
from IPython.display import display
import matplotlib.pyplot as plt
from functools import partial
%matplotlib inline

Code borrow from [eriklindernoren](https://github.com/eriklindernoren), [fchollet](https://github.com/fchollet) and [keras-contrib](https://github.com/keras-team/keras-contrib)

https://github.com/eriklindernoren/Keras-GAN/blob/master/aae/adversarial_autoencoder.py

https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.5-introduction-to-gans.ipynb

https://github.com/keras-team/keras-contrib/blob/master/examples/improved_wgan.py

In [8]:
class FaceSwapGAN():
    def __init__(self, batch_size=8, img_dirA='./faceA/*.*', img_dirB='./faceB/*.*', use_mixup=False):
        self.img_size = 64 
        self.channels = 3
        self.img_shape = (self.img_size, self.img_size, self.channels)
        self.batch_size = batch_size
        self.img_dirA = img_dirA
        self.img_dirB = img_dirB
        self.random_transform_args = {
            'rotation_range': 20,
            'zoom_range': 0.05,
            'shift_range': 0.05,
            'random_flip': 0.5,
            }
        self.use_mixup = use_mixup 
        self.mixup_alpha = 0.2        
        self.n_critic = 5
        if self.use_mixup:
            rand_wavg_batch_size = self.batch_size
        else:
            rand_wavg_batch_size = self.batch_size * 2
        optimizer = Adam(1e-4, 0.5, 0.9)
        
        def wasserstein_loss(y_true, y_pred):
            return K.mean(y_true * y_pred)

        def gradient_penalty_loss(y_true, y_pred, averaged_samples, gradient_penalty_weight):
            gradients = K.gradients(K.sum(y_pred), averaged_samples)
            gradient_l2_norm = K.sqrt(K.sum(K.square(gradients)))
            gradient_penalty = gradient_penalty_weight * K.square(1 - gradient_l2_norm)
            return gradient_penalty
    
        class RandomWeightedAverage(_Merge):
            def _merge_function(self, inputs):
                weights = K.random_uniform((rand_wavg_batch_size, 1, 1, 1))
                return (weights * inputs[0]) + ((1 - weights) * inputs[1])

        # Build and compile the discriminator
        self.netDA, self.netDB = self.build_discriminator()
        self.netDA.compile(loss=wasserstein_loss, optimizer=optimizer, metrics=['accuracy'])
        self.netDB.compile(loss=wasserstein_loss, optimizer=optimizer, metrics=['accuracy'])

        # Build and compile the generator
        self.netGA, self.netGB = self.build_generator()
        try:
            self.netGA.load_weights("models/netGA.h5")
            self.netGB.load_weights("models/netGB.h5")
            print ("Generator models loaded.")
        except:
            print ("Generator weights files not found.")
            pass
        self.netGA.compile(loss=['mae', wasserstein_loss], optimizer=optimizer)
        self.netGB.compile(loss=['mae', wasserstein_loss], optimizer=optimizer)  

        warped_img = Input(shape=self.img_shape)
        real_img = Input(shape=self.img_shape)
        alphaA, reconstructed_imgA = self.netGA(warped_img)
        alphaB, reconstructed_imgB = self.netGB(warped_img)             

        # For the adversarial_autoencoder model we will only train the generator
        self.netDA.trainable = False
        self.netDB.trainable = False

        def one_minus(x): return 1 - x
        # masked_img = alpha * reconstructed_img + (1 - alpha) * img
        masked_imgA = add([multiply([alphaA, reconstructed_imgA]), 
                           multiply([Lambda(one_minus)(alphaA), warped_img])])
        masked_imgB = add([multiply([alphaB, reconstructed_imgB]), 
                           multiply([Lambda(one_minus)(alphaB), warped_img])])
        out_discriminatorA = self.netDA(concatenate([masked_imgA, warped_img], axis=-1))
        out_discriminatorB = self.netDB(concatenate([masked_imgB, warped_img], axis=-1))

        # The adversarial_autoencoder model  (stacked generator and discriminator) takes
        # img as input => generates encoded represenation and reconstructed image => determines validity 
        self.adversarial_autoencoderA = Model(warped_img, [reconstructed_imgA, out_discriminatorA, alphaA])
        self.adversarial_autoencoderB = Model(warped_img, [reconstructed_imgB, out_discriminatorB, alphaB])
        self.adversarial_autoencoderA.compile(loss=['mae', wasserstein_loss, 'mse'],
                                              loss_weights=[1, .5, 3e-3],
                                              optimizer=optimizer)
        self.adversarial_autoencoderB.compile(loss=['mae', wasserstein_loss, 'mse'],
                                              loss_weights=[1, .5, 3e-3],
                                              optimizer=optimizer)
        
        # Setting trainable=True for discriminators
        # If not set, keras will throw error: Non type not supported (due to missing gradient from discriminator)
        self.netDA.trainable = True
        self.netDB.trainable = True
        self.netGA.trainable = False
        self.netGB.trainable = False
        averaged_imgA = RandomWeightedAverage()([real_img, reconstructed_imgA])
        averaged_imgB = RandomWeightedAverage()([real_img, reconstructed_imgB])
        averaged_outA = self.netDA(concatenate([averaged_imgA, warped_img], axis=-1))
        averaged_outB = self.netDB(concatenate([averaged_imgB, warped_img], axis=-1))
        partial_gp_lossA = partial(gradient_penalty_loss, 
                                   averaged_samples=averaged_imgA, 
                                   gradient_penalty_weight=10.)
        partial_gp_lossA.__name__ = 'gradient_penaltyA'
        partial_gp_lossB = partial(gradient_penalty_loss, 
                                   averaged_samples=averaged_imgB, 
                                   gradient_penalty_weight=10.)
        partial_gp_lossB.__name__ = 'gradient_penaltyB'
        self.netDA = Model(inputs=[self.netDA.inputs[0], real_img, warped_img],
                           outputs=[self.netDA.outputs[0], averaged_outA])
        self.netDB = Model(inputs=[self.netDB.inputs[0], real_img, warped_img],
                           outputs=[self.netDB.outputs[0], averaged_outB])
        try:
            self.netDA.load_weights("models/netDA.h5") 
            self.netDB.load_weights("models/netDB.h5") 
            print ("Discriminator models loaded.")
        except:
            print ("Discriminator weights files not found.")
            pass
        self.netDA.compile(optimizer=optimizer, loss=[wasserstein_loss, partial_gp_lossA])
        self.netDB.compile(optimizer=optimizer, loss=[wasserstein_loss, partial_gp_lossB]) 
        

    def build_generator(self):
        def conv_block(input_tensor, f):
            x = input_tensor
            x = Conv2D(f, kernel_size=3, strides=2, kernel_initializer=RandomNormal(0, 0.02), 
                       use_bias=False, padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            return x

        def res_block(input_tensor, f):
            x = input_tensor
            x = Conv2D(f, kernel_size=3, kernel_initializer=RandomNormal(0, 0.02), 
                       use_bias=False, padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = Conv2D(f, kernel_size=3, kernel_initializer=RandomNormal(0, 0.02), 
                       use_bias=False, padding="same")(x)
            x = add([x, input_tensor])
            x = LeakyReLU(alpha=0.2)(x)
            return x

        def upscale_ps(filters, use_norm=True):
            def block(x):
                x = Conv2D(filters*4, kernel_size=3, use_bias=False, 
                           kernel_initializer=RandomNormal(0, 0.02), padding='same' )(x)
                x = LeakyReLU(0.1)(x)
                x = PixelShuffler()(x)
                return x
            return block

        def Encoder(img_shape):
            inp = Input(shape=img_shape)
            x = Conv2D(64, kernel_size=5, kernel_initializer=RandomNormal(0, 0.02), 
                       use_bias=False, padding="same")(inp)
            x = conv_block(x,128)
            x = conv_block(x,256)
            x = conv_block(x,512) 
            x = conv_block(x,1024)
            x = Dense(1024)(Flatten()(x))
            x = Dense(4*4*1024)(x)
            x = Reshape((4, 4, 1024))(x)
            out = upscale_ps(512)(x)
            return Model(inputs=inp, outputs=out)

        def Decoder_ps(img_shape):
            nc_in = 512
            input_size = img_shape[0]//8
            inp = Input(shape=(input_size, input_size, nc_in))
            x = inp
            x = upscale_ps(256)(x)
            x = upscale_ps(128)(x)
            x = upscale_ps(64)(x)
            x = res_block(x, 64)
            x = res_block(x, 64)
            alpha = Conv2D(1, kernel_size=5, padding='same', activation="sigmoid")(x)
            rgb = Conv2D(3, kernel_size=5, padding='same', activation="tanh")(x)
            return Model(inp, [alpha, rgb])
        
        encoder = Encoder(self.img_shape)
        decoder_A = Decoder_ps(self.img_shape)
        decoder_B = Decoder_ps(self.img_shape)    
        x = Input(shape=self.img_shape)
        netGA = Model(x, decoder_A(encoder(x)))
        netGB = Model(x, decoder_B(encoder(x)))         
        return netGA, netGB, 

    def build_discriminator(self):  
        def conv_block_d(input_tensor, f, use_instance_norm=True):
            x = input_tensor
            x = Conv2D(f, kernel_size=4, strides=2, kernel_initializer=RandomNormal(0, 0.02), 
                       use_bias=False, padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            return x   
        def Discriminator(img_shape):
            inp = Input(shape=(img_shape[0], img_shape[1], img_shape[2]*2))
            x = conv_block_d(inp, 64, False)
            x = conv_block_d(x, 128, False)
            x = conv_block_d(x, 256, False)
            out = Conv2D(1, kernel_size=4, kernel_initializer=RandomNormal(0, 0.02), 
                         use_bias=False, padding="same", activation="sigmoid")(x)   
            return Model(inputs=[inp], outputs=out) 
        
        netDA = Discriminator(self.img_shape)
        netDB = Discriminator(self.img_shape)       
        return netDA, netDB    


    def train(self, max_iters, save_interval=50):        
        def load_data(file_pattern):
            return glob.glob(file_pattern)
        
        def read_image(fn, random_transform_args=self.random_transform_args):
            image = cv2.imread(fn)
            image = cv2.resize(image, (256,256)) / 255 * 2 - 1
            image = random_transform(image, **random_transform_args )
            warped_img, target_img = random_warp(image)
            return warped_img, target_img

        def minibatch(data, batchsize):
            length = len(data)
            epoch = i = 0
            tmpsize = None  
            shuffle(data)
            while True:
                size = tmpsize if tmpsize else batchsize
                if i+size > length:
                    shuffle(data)
                    i = 0
                    epoch+=1        
                rtn = np.float32([read_image(data[j]) for j in range(i,i+size)])
                i+=size
                tmpsize = yield epoch, rtn[:,0,:,:,:], rtn[:,1,:,:,:]       

        def minibatchAB(dataA, batchsize):
            batchA = minibatch(dataA, batchsize)
            tmpsize = None    
            while True:        
                ep1, warped_img, target_img = batchA.send(tmpsize)
                tmpsize = yield ep1, warped_img, target_img

        batch_size = self.batch_size    
            
        # Load the dataset
        train_A = load_data(self.img_dirA)
        train_B = load_data(self.img_dirB)        
        assert len(train_A), "No image found in " + str(img_dirA) + "."
        assert len(train_B), "No image found in " + str(img_dirB) + "."
        train_batchA = minibatchAB(train_A, batch_size)
        train_batchB = minibatchAB(train_B, batch_size)

        print ("Training starts...")
        t0 = time.time()
        gen_iterations = 0
        while gen_iterations < max_iters:
            #print ("iter: " + str(gen_iterations))

            # ---------------------
            #  Train Discriminators
            # ---------------------

            # Select a random half batch of images
            epoch, warped_A, target_A = next(train_batchA) 
            epoch, warped_B, target_B = next(train_batchB) 

            # Generate a half batch of new images
            gen_alphasA, gen_imgsA = self.netGA.predict(warped_A)
            gen_alphasB, gen_imgsB = self.netGB.predict(warped_B)
            #gen_masked_imgsA = gen_alphasA * gen_imgsA + (1 - gen_alphasA) * warped_A
            #gen_masked_imgsB = gen_alphasB * gen_imgsB + (1 - gen_alphasB) * warped_B
            gen_masked_imgsA = np.array([gen_alphasA[i] * gen_imgsA[i] + (1 - gen_alphasA[i]) * warped_A[i] 
                                         for i in range(batch_size)])
            gen_masked_imgsB = np.array([gen_alphasB[i] * gen_imgsB[i] + (1 - gen_alphasB[i]) * warped_B[i]
                                         for i in range (batch_size)])

            positive_y = np.ones((batch_size, ) + self.netDA.output_shape[0][1:])
            negative_y = -positive_y #np.zeros((batch_size, ) + self.netDA.output_shape[0][1:])
            gp_loss_zeros = np.zeros((batch_size, ) + self.netDA.output_shape[0][1:])
            
            concat_real_inputA = np.array([np.concatenate([target_A[i], warped_A[i]], axis=-1) 
                                           for i in range(batch_size)])
            concat_real_inputB = np.array([np.concatenate([target_B[i], warped_B[i]], axis=-1) 
                                           for i in range(batch_size)])
            concat_fake_inputA = np.array([np.concatenate([gen_masked_imgsA[i], warped_A[i]], axis=-1) 
                                           for i in range(batch_size)])
            concat_fake_inputB = np.array([np.concatenate([gen_masked_imgsB[i], warped_B[i]], axis=-1) 
                                           for i in range(batch_size)])
            if self.use_mixup:
                lam = np.random.beta(self.mixup_alpha, self.mixup_alpha)
                mixup_A = lam * concat_real_inputA + (1 - lam) * concat_fake_inputA
                mixup_B = lam * concat_real_inputB + (1 - lam) * concat_fake_inputB
                mixup_label = lam * positive_y + (1 - lam) * negative_y

            # Train the discriminators
            #print ("Train the discriminators.")
            if self.use_mixup:
                d_lossA = self.netDA.train_on_batch([mixup_A, target_A, warped_A], [mixup_label, gp_loss_zeros])
                d_lossB = self.netDB.train_on_batch([mixup_B, target_B, warped_B], [mixup_label, gp_loss_zeros])
            else:
                d_lossA = self.netDA.train_on_batch([np.concatenate([concat_real_inputA, concat_fake_inputA], axis=0),
                                                     np.concatenate([target_A, target_A], axis=0), 
                                                     np.concatenate([warped_A, warped_A], axis=0)], 
                                                    [np.concatenate([positive_y, negative_y], axis=0), 
                                                     np.concatenate([gp_loss_zeros, gp_loss_zeros], axis=0)])
                d_lossB = self.netDB.train_on_batch([np.concatenate([concat_real_inputB, concat_fake_inputB], axis=0),
                                                     np.concatenate([target_B, target_B], axis=0), 
                                                     np.concatenate([warped_B, warped_B], axis=0)], 
                                                    [np.concatenate([positive_y, negative_y], axis=0), 
                                                     np.concatenate([gp_loss_zeros, gp_loss_zeros], axis=0)])


            # ---------------------
            #  Train Generators
            # ---------------------

            # Train the generators
            #print ("Train the generators.")
            if (gen_iterations + 1) % self.n_critic == 0:
                mask_regularizationA = np.zeros((batch_size, ) + self.netGA.output_shape[0][1:])
                mask_regularizationB = np.zeros((batch_size, ) + self.netGB.output_shape[0][1:])
                g_lossA = self.adversarial_autoencoderA.train_on_batch(warped_A, 
                                                                       [target_A, positive_y, mask_regularizationA])
                g_lossB = self.adversarial_autoencoderB.train_on_batch(warped_B, 
                                                                       [target_B, positive_y, mask_regularizationB])           
            gen_iterations += 1             

            # If at save interval => save models & show results
            if (gen_iterations) % save_interval == 0:
                clear_output()
                # Plot the progress
                print('[%d/%s][%d] Loss_DA: %f Loss_DB: %f Loss_GA: %f Loss_GB: %f time: %f'
                      % (epoch, "num_epochs", gen_iterations, d_lossA[0], 
                         d_lossB[0], g_lossA[0], g_lossB[0], time.time()-t0)) 
                
                # Save models
                self.netGA.save_weights("models/netGA.h5")
                self.netGB.save_weights("models/netGB.h5" )
                self.netDA.save_weights("models/netDA.h5")
                self.netDB.save_weights("models/netDB.h5")
                print ("Models saved.")
                
                # Show results
                _, wA, tA = train_batchA.send(14)  
                _, wB, tB = train_batchB.send(14)
                self.showG(tA, tB)
            
    def showG(self, test_A, test_B):      
        def display_fig(figure_A, figure_B):
            figure = np.concatenate([figure_A, figure_B], axis=0 )
            figure = figure.reshape((4,7) + figure.shape[1:])
            figure = stack_images(figure)
            figure = np.clip((figure + 1) * 255 / 2, 0, 255).astype('uint8')
            figure = cv2.cvtColor(figure, cv2.COLOR_BGR2RGB)
            display(Image.fromarray(figure)) 
            
        out_test_A_netGA = self.netGA.predict(test_A)
        out_test_A_netGB = self.netGB.predict(test_A)
        out_test_B_netGA = self.netGA.predict(test_B)
        out_test_B_netGB = self.netGB.predict(test_B)
        
        figure_A = np.stack([
            test_A,
            out_test_A_netGA[0] * out_test_A_netGA[1] + (1 - out_test_A_netGA[0]) * test_A,
            out_test_A_netGB[0] * out_test_A_netGB[1] + (1 - out_test_A_netGB[0]) * test_A,
            ], axis=1 )
        figure_B = np.stack([
            test_B,
            out_test_B_netGB[0] * out_test_B_netGB[1] + (1 - out_test_B_netGB[0]) * test_B,
            out_test_B_netGA[0] * out_test_B_netGA[1] + (1 - out_test_B_netGA[0]) * test_B,
            ], axis=1 )
        print ("Masked results:")
        display_fig(figure_A, figure_B)   
        
        figure_A = np.stack([
            test_A,
            out_test_A_netGA[1],
            out_test_A_netGB[1],
            ], axis=1 )
        figure_B = np.stack([
            test_B,
            out_test_B_netGB[1],
            out_test_B_netGA[1],
            ], axis=1 )
        print ("Raw results:")
        display_fig(figure_A, figure_B)       
        
        figure_A = np.stack([
            test_A,
            np.tile(out_test_A_netGA[0],3) * 2 - 1,
            np.tile(out_test_A_netGB[0],3) * 2 - 1,
            ], axis=1 )
        figure_B = np.stack([
            test_B,
            np.tile(out_test_B_netGB[0],3) * 2 - 1,
            np.tile(out_test_B_netGA[0],3) * 2 - 1,
            ], axis=1 )
        print ("Alpha masks:")
        display_fig(figure_A, figure_B)        

In [10]:
!mkdir models

mkdir: cannot create directory ‘models’: File exists


In [9]:
gan = FaceSwapGAN()

Generator models loaded.
Discriminator models loaded.


In [ ]:
gan.train(max_iters=10e4, save_interval=500)

## Video

In [14]:
import face_recognition
from moviepy.editor import VideoFileClip

In [41]:
use_smoothed_mask = True
use_smoothed_bbox = True


def get_smoothed_coord(x0, x1, y0, y1):
    global prev_x0, prev_x1, prev_y0, prev_y1
    x0 = int(0.65*prev_x0 + 0.35*x0)
    x1 = int(0.65*prev_x1 + 0.35*x1)
    y1 = int(0.65*prev_y1 + 0.35*y1)
    y0 = int(0.65*prev_y0 + 0.35*y0)
    return x0, x1, y0, y1    
    
def set_global_coord(x0, x1, y0, y1):
    global prev_x0, prev_x1, prev_y0, prev_y1
    prev_x0 = x0
    prev_x1 = x1
    prev_y1 = y1
    prev_y0 = y0

def process_video(input_img):   
    #input_img = input_img[:, input_img.shape[1]//3:2*input_img.shape[1]//3,:]
    input_img = input_img[:, :640,:]
    image = input_img
    faces = face_recognition.face_locations(image, model="cnn")
    
    if len(faces) == 0:
        comb_img = np.zeros([input_img.shape[0], input_img.shape[1]*2,input_img.shape[2]])
        comb_img[:, :input_img.shape[1], :] = input_img
        comb_img[:, input_img.shape[1]:, :] = input_img
        triple_img = np.zeros([input_img.shape[0], input_img.shape[1]*3,input_img.shape[2]])
        triple_img[:, :input_img.shape[1], :] = input_img
        triple_img[:, input_img.shape[1]:input_img.shape[1]*2, :] = input_img      
        triple_img[:, input_img.shape[1]*2:, :] = (input_img * .15).astype('uint8')
    
    mask_map = np.zeros_like(image)
    
    global prev_x0, prev_x1, prev_y0, prev_y1
    global frames    
    for (x0, y1, x1, y0) in faces:
        h = x1 - x0
        w = y1 - y0
        
        # smoothing bounding box
        if use_smoothed_bbox:
            if frames != 0:
                x0, x1, y0, y1 = get_smoothed_coord(x0, x1, y0, y1)
                set_global_coord(x0, x1, y0, y1)
            else:
                set_global_coord(x0, x1, y0, y1)
                frames += 1
            
        cv2_img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        roi_image = cv2_img[x0+h//15:x1-h//15,y0+w//15:y1-w//15,:]
        roi_size = roi_image.shape  
        
        # smoothing mask
        if use_smoothed_mask:
            mask = np.zeros_like(roi_image)
            mask[h//15:-h//15,w//15:-w//15,:] = 255
            mask = cv2.GaussianBlur(mask,(15,15),10)
            orig_img = cv2.cvtColor(roi_image, cv2.COLOR_BGR2RGB)
        
        ae_input = cv2.resize(roi_image, (64,64))/255. * 2 - 1        
        result = gan.netGA.predict(np.array([ae_input])) # Change path_A/path_B here
        result_a = result[0][0] * 255
        result_bgr = np.clip( (result[1][0] + 1) * 255 / 2, 0, 255 )
        result_a = cv2.GaussianBlur(result_a ,(7,7),6)
        result_a = np.expand_dims(result_a, axis=2)
        result = (result_a/255 * result_bgr + (1 - result_a/255) * ((ae_input + 1) * 255 / 2)).astype('uint8')
        result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
        
        mask_map[x0+h//15:x1-h//15, y0+w//15:y1-w//15,:] = np.expand_dims(cv2.resize(result_a, (roi_size[1],roi_size[0])), axis=2)
        mask_map = np.clip(mask_map + .15 * input_img, 0, 255 )
        
        result = cv2.resize(result, (roi_size[1],roi_size[0]))
        comb_img = np.zeros([input_img.shape[0], input_img.shape[1]*2,input_img.shape[2]])
        comb_img[:, :input_img.shape[1], :] = input_img
        comb_img[:, input_img.shape[1]:, :] = input_img
        
        if use_smoothed_mask:
            comb_img[x0+h//15:x1-h//15, input_img.shape[1]+y0+w//15:input_img.shape[1]+y1-w//15,:] = mask/255*result + (1-mask/255)*orig_img
        else:
            comb_img[x0+h//15:x1-h//15, input_img.shape[1]+y0+w//15:input_img.shape[1]+y1-w//15,:] = result
            
        triple_img = np.zeros([input_img.shape[0], input_img.shape[1]*3,input_img.shape[2]])
        triple_img[:, :input_img.shape[1]*2, :] = comb_img
        triple_img[:, input_img.shape[1]*2:, :] = mask_map
    
    return triple_img#comb_img

In [ ]:
# Variables for smoothing bounding box
global prev_x0, prev_x1, prev_y0, prev_y1
global frames
prev_x0 = prev_x1 = prev_y0 = prev_y1 = 0
frames = 0

output = 'OUTPUT_VIDEO.mp4'
clip1 = VideoFileClip("INPUT_VIDEO.mp4")
clip = clip1.fl_image(process_video)#.subclip(11, 13) #NOTE: this function expects color images!!
%time clip.write_videofile(output, audio=False)